In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.monitor import Monitor
from yawning_titan.envs.generic.core.blue_interface import BlueInterface
from yawning_titan.envs.generic.core.network_interface import NetworkInterface
from yawning_titan.envs.generic.core.red_interface import RedInterface
from yawning_titan.envs.generic.generic_env import GenericNetworkEnv
from yawning_titan.game_modes.game_mode_db import default_game_mode
from yawning_titan.networks.network_db import default_18_node_network

In [ ]:
import logging

logging.basicConfig(format="%(asctime)s - %(name)s - %(levelname)s - %(message)s")
logger = logging.getLogger()

In [ ]:
game_mode = default_game_mode()

In [ ]:
blue_dict = game_mode.to_dict()["blue"]["action_set"]

for key in blue_dict.keys():
    print(key, blue_dict[key])
    print("-----------------------")

In [ ]:
network = default_18_node_network()

In [ ]:
network_interface = NetworkInterface(game_mode=game_mode, network=network)

In [ ]:
red = RedInterface(network_interface)
blue = BlueInterface(network_interface)

In [ ]:
env = GenericNetworkEnv(red, blue, network_interface)

In [ ]:
check_env(env, warn=True)

_ = env.reset()

In [ ]:
from aegis.llm_agent import LLMAgent, LLMPolicy
from text_generation import Client

agent = LLMAgent(env=env, policy=LLMPolicy, client=Client(base_url="http://192.168.0.8:58084"))

In [ ]:
# from aegis.random_agent import RandomAgent, RandomPolicy

# agent = RandomAgent(env=env, policy=RandomPolicy)

In [ ]:
obs = env.reset()
action, _ = agent.predict(obs)
obs, reward, done, info = env.step(action=int(action))

In [ ]:
from aegis.evaluate import evaluate_policy

rm, rstd = evaluate_policy(agent, Monitor(env), n_eval_episodes=3, return_episode_rewards=False)

In [ ]:
print(f"Mean reward: {rm:.4f}\nStd Reward: {rstd:.4f}")